# Create elevation layers for Mapbox

![Contours for import to Mapbox](graphics/qa_15.jpg)

In [22]:
# Import ArcGIS package
import arcpy
# Subprocess allows us to issue commands on the command line
import subprocess
# Zip utlity to extract files
from zipfile import ZipFile
# zip up a directory
import shutil

In [23]:
# Directory for project 
# Got to use double backslash because of other modules
out_directory = "C:\\MontgomeryGIS\\final\\CraftCreekAnalysis\\"

# Project name - creates a folder in project directory
project = "craftcreek"

# Output geodatabase name
out_geodb = "workspaceCC.gdb"
out_shp = "shapefiles"

# Input layers
in_dem = "C:\MontgomeryGIS\craftcreek\craftcreek.gdb\craftcreek_dem_5ft"
aoi = "C:\MontgomeryGIS\craftcreek\craftcreek.gdb\craftcreek_4000ft"

# Output parameters
contour_interval = 10
contour_index = 100
contour_name = f"contour_{contour_interval}ft"


In [24]:
# Create folders
subprocess.run(f'mkdir {out_directory}{project}', shell=True)
subprocess.run(f'mkdir {out_directory}{project}\\{out_shp}', shell=True)
completed = subprocess.run(f'dir {project}', shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))

 Volume in drive C is Windows
 Volume Serial Number is 44BA-8B6A

 Directory of c:\MontgomeryGIS\final\CraftCreekAnalysis\craftcreek

12/11/2019  06:54 PM    <DIR>          .
12/11/2019  06:54 PM    <DIR>          ..
12/11/2019  06:54 PM    <DIR>          shapefiles
               0 File(s)              0 bytes
               3 Dir(s)  15,443,185,664 bytes free



In [25]:
arcpy.env.overwriteOutput = True

In [26]:
# Create project geodatabase
arcpy.CreateFileGDB_management(f'{out_directory}{project}', out_geodb)

<Result 'C:\\MontgomeryGIS\\final\\CraftCreekAnalysis\\craftcreek\\workspaceCC.gdb'>

In [27]:
# Create project default geodatabase
out_path = f'{out_directory}{project}\\{out_geodb}'
arcpy.env.workspace = out_path
completed = subprocess.run(f'dir {project}', shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))

 Volume in drive C is Windows
 Volume Serial Number is 44BA-8B6A

 Directory of c:\MontgomeryGIS\final\CraftCreekAnalysis\craftcreek

12/11/2019  06:54 PM    <DIR>          .
12/11/2019  06:54 PM    <DIR>          ..
12/11/2019  06:54 PM    <DIR>          shapefiles
12/11/2019  06:54 PM    <DIR>          workspaceCC.gdb
               0 File(s)              0 bytes
               4 Dir(s)  15,442,403,328 bytes free



In [28]:
neighborhood = arcpy.sa.NbrCircle(3,'CELL')
outFocalStat = arcpy.sa.FocalStatistics(in_dem, neighborhood, "MEAN")
outFocalStat.save("avg_elev_30ft")
arcpy.Contour_3d ("avg_elev_30ft", contour_name, contour_interval)

<Result 'C:\\MontgomeryGIS\\final\\CraftCreekAnalysis\\craftcreek\\workspaceCC.gdb\\contour_10ft'>

In [51]:
code_block = """
def is_index_contour(x):
    if x%contour_index == 0:
        return 1
    else:
        return 0
"""     
arcpy.AddField_management (contour_name, "index", "SHORT")        
arcpy.CalculateField_management(contour_name, "index", "is_index_contour(!CONTOUR!)", "PYTHON3", code_block)

<Result 'C:\\MontgomeryGIS\\SkyBridge\\SkyBridge\\workspace.gdb\\contour_10ft'>

In [29]:
arcpy.Clip_analysis (contour_name, aoi, f'{out_directory}{project}\\{out_shp}\\{contour_name}_aoi.shp')

<Result 'C:\\MontgomeryGIS\\final\\CraftCreekAnalysis\\craftcreek\\shapefiles\\contour_10ft_aoi.shp'>

In [33]:
zipfile = ZipFile(f"{out_directory}{project}\\{contour_name}_aoi.zip", "w" )
zipfile.write(f'{out_directory}{project}\\{out_shp}\\', compress_type=zipfile.ZIP_STORED)

AttributeError: 'ZipFile' object has no attribute 'ZIP_STORED'

In [32]:
shutil.make_archive(f"{out_directory}{project}\\{contour_name}_aoi", 'zip', f'{out_directory}{project}\\{out_shp}\\')

'C:\\MontgomeryGIS\\final\\CraftCreekAnalysis\\craftcreek\\contour_10ft_aoi.zip'

In [21]:
subprocess.run(f'del {out_directory}{project}\\{out_shp}\\*.*', shell=True)

CompletedProcess(args='del C:\\MontgomeryGIS\\final(craft-creek data)\\craftcreek\\shapefiles\\*.*', returncode=1)